# 1) Introduction: Computational Geometry and scipy.spatial

In [6]:
import scipy
import numpy as np

# 2) Distances between arrays of points

## 2.1) scipy.spatial.distance_matrix

### 2.1.1) Euclidean (L2 norm)

There are many cases where it is desirable to calculate the distances between all possible pairs of points from two data sets. Consider a self-driving vehicle that is programmed to avoid animals that enter a roadway. It may have a set of coordinates that represent the vehicle proper relative to some kind of camera / detector, and another set of coordinates that represent the apparent obstruction (animal on the road).

For simplicity, lets define the two sets of coordinates randomly:

In [8]:
vehicle_coords = np.random.random((50,3))
animal_coords = np.random.random((600,3))

In order to calculate the amount of time that the vehicle has to make a course adjustment, it would be sensible to first calculate the closest distance between a point on the vehicle and a point on the animal. Lets try this by calculating all possible distances between the objects and finding the minimum of that distance matrix.

In [11]:
from scipy.spatial import distance_matrix
all_distances = distance_matrix(vehicle_coords, animal_coords)
all_distances.shape

(50, 600)

Each row of the distance matrix represents a point on the vehicle, and each column (value in a given row) is the distance between that point on the vehicle and one of the points on the animal (ordered by index).

In [13]:
closest_distance = all_distances.min()
closest_distance

0.028003950663311409

### 2.1.2) Rectilinear (L1 norm)

Consider another situation, where a self-driving vehicle is aware (i.e., via real-time 'traffic' reporting) of an animal obstructing a roadway, even though the vehicle does not have line of sight to the animal obstacle. The animal is a few city blocks away, but the vehicle would still benefit from a distance & time estimate so that appropriate adjustments may be made. 

In this case, the straight-line Euclidean distance would not represent the shortest distance the vehicle would travel in practice--instead, we need to consider the so-called L1 norm / taxicab geometry/ Manhattan distance, etc. 

In [16]:
taxicab_distances = distance_matrix(vehicle_coords, animal_coords, p=1)
taxicab_distances.shape

(50, 600)

In [17]:
closest_taxicab_distance = taxicab_distances.min()
closest_taxicab_distance

0.039965928608402312